In [1]:
import os
import shutil

!hostname

cheyenne6


In [2]:
os.environ['PROJECT'] = 'P93300324'

casename = 'b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301'
cesm_dir = '/glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu'
case_dir = f'/glade/u/home/fengzhu/CESM/cases/{casename}'
output_root = f'/glade/scratch/fengzhu/CESM_output'
output_dir = os.path.join(output_root, casename)

print('case_dir', case_dir)
print('output_dir', output_dir)

shutil.rmtree(case_dir) if os.path.exists(case_dir) else None
shutil.rmtree(output_dir) if os.path.exists(output_dir) else None

case_dir /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301
output_dir /glade/scratch/fengzhu/CESM_output/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301


In [3]:
!$cesm_dir/cime/scripts/create_newcase --case $case_dir --res ne16_g17 --machine cheyenne --compset 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV --output-root $output_root --run-unsupported || exit 1

Compset longname is 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV
Compset specification file is /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime_config/config_compsets.xml
Compset forcing is 1850
ATM component is CAM cam5 physics:
LND component is clm4.0:Satellite phenology:
ICE component is Sea ICE (cice) model version 4
OCN component is POP2 
ROF component is RTM: River Transport Model
GLC component is Stub glacier (land ice) component
WAV component is Stub wave component
ESP component is 
Pes     specification file is /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime_config/config_pes.xml
Compset specific settings: name is RUN_STARTDATE and value is 0001-01-01
Machine is cheyenne
Pes setting: grid          is a%ne16np4_l%ne16np4_oi%gx1v7_r%r05_g%null_w%null_m%gx1v7 
Pes setting: compset       is 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV 
Pes setting: tasks       is {'NTASKS_ATM': -3, 'NTASKS_LND': -1, 'NTASKS_ROF': -1, 'NTASKS_ICE': -2, 'NTASKS_OCN': -1, 'NTASKS_GLC

In [4]:
!cp /glade/p/cesm/palwg/paleoweather/cesm_tags/mods/cam_diagnostics.F90   $case_dir/SourceMods/src.cam/
!cp $cesm_dir/components/clm/src_clm40/biogeophys/BalanceCheckMod.F90   $case_dir/SourceMods/src.clm/
!sed -i '817 s/^/\!/' $case_dir/SourceMods/src.clm/BalanceCheckMod.F90
!sed -i '818 s/^/\!/' $case_dir/SourceMods/src.clm/BalanceCheckMod.F90

In [5]:
os.chdir(case_dir)
os.getcwd()

'/glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301'

In [6]:
!./xmlchange RUN_TYPE=startup,GET_REFCASE=FALSE
!./xmlchange CAM_CONFIG_OPTS="-water_tracer h2o_h216o_hdo_h218o" --append
!./xmlchange OCN_TRACER_MODULES="iage wiso"
!./xmlchange FLDS_WISO=TRUE
!./xmlchange CLM_FORCE_COLDSTART=on
!./xmlchange CLM_BLDNML_OPTS="-ignore_warnings" --append
!./xmlchange OCN_NCPL=1,CPL_SEQ_OPTION=CESM1_MOD  # daily coupling

ntasks_atm = 1536
ntasks_ocn = 256
ntasks_lnd = 512
ntasks_ice = 1024
!./xmlchange NTASKS_CPL=$ntasks_atm,NTHRDS_CPL=1,ROOTPE_CPL=0
!./xmlchange NTASKS_ATM=$ntasks_atm,NTHRDS_ATM=1,ROOTPE_ATM=0
!./xmlchange NTASKS_LND=$ntasks_lnd,NTHRDS_LND=1,ROOTPE_LND=0
!./xmlchange NTASKS_ICE=$ntasks_ice,NTHRDS_ICE=1,ROOTPE_ICE=$ntasks_lnd
!./xmlchange NTASKS_OCN=$ntasks_ocn,NTHRDS_OCN=1,ROOTPE_OCN=$ntasks_atm
!./xmlchange NTASKS_ROF=$ntasks_lnd,NTHRDS_ROF=1,ROOTPE_ROF=0

In [7]:
gridmap_dir = '/glade/p/cesmdata/inputdata/cpl/gridmaps/'
!./xmlchange ATM2OCN_FMAPNAME=$gridmap_dir/ne16np4/map_ne16np4_TO_gx1v7_aave.171018.nc
!./xmlchange ATM2OCN_SMAPNAME=$gridmap_dir/ne16np4/map_ne16np4_TO_gx1v7_blin.171018.nc
!./xmlchange ATM2OCN_VMAPNAME=$gridmap_dir/ne16np4/map_ne16np4_TO_gx1v7_patc.171018.nc
!./xmlchange OCN2ATM_FMAPNAME=$gridmap_dir/gx1v7/map_gx1v7_TO_ne16np4_aave.171018.nc
!./xmlchange OCN2ATM_SMAPNAME=$gridmap_dir/gx1v7/map_gx1v7_TO_ne16np4_aave.171018.nc
# !./xmlchange LND2ROF_FMAPNAME=$gridmap_dir/
# !./xmlchange ROF2LND_FMAPNAME=$gridmap_dir/
# !./xmlchange ROF2OCN_FMAPNAME=$gridmap_dir/
!./xmlchange ROF2OCN_RMAPNAME=$gridmap_dir/r05/map_r05_to_gx1v7_nnsm_e1000r300_170413.nc

In [8]:
!./case.setup

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
copying /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime/config/cesm/machines/Depends.intel -> /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301
/glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/env_mach_specific.xml already exists, delete to replace
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None
Creating batch scripts
Writing case.run script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
copying /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/env_mach_pes.xml -> /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/LockedFi

In [9]:
with open('./user_nl_cam', 'a') as f:
    f.write('''
        cldfrc_rhminl  = 0.8800D0
        micro_mg_dcs   = 400.D-6
        dust_emis_fact = 0.95D0
        wtrc_limiter_phis_crit = 100000.0
    ''')

In [10]:
with open('./user_nl_pop', 'a') as f:
    f.write('''
        ! namelists for wiso
        wiso_use_nml_surf_vals      = .false.
        wiso_comp_surf_avg_freq_opt = 'nmonth'
        wiso_comp_surf_avg_freq     = 1
        init_ts_suboption       = 'spunup'
        init_ts_file_fmt        = 'nc'
        init_ts_file            = '/glade/p/cgd/ppc/jiangzhu/rest/b.e13.Bi1850C5.f19_g16.05/b.e13.Bi1850C5.f19_g16.05.pop.r.0505-01-01-00000.nc'
    ''')

In [11]:
with open('./user_nl_cice', 'a') as f:
    f.write('''
        ice_ic = '/glade/p/cgd/ppc/jiangzhu/rest/b.e13.Bi1850C5.f19_g16.05/b.e13.Bi1850C5.f19_g16.05.cice.r.0505-01-01-00000.nc'
    ''')

In [12]:
with open('./user_nl_rtm', 'a') as f:
    f.write('''
        finidat_rtm = '/glade/p/cgd/ppc/jiangzhu/rest/b.e13.Bi1850C5.f19_g16.05/b.e13.Bi1850C5.f19_g16.05.rtm.r.0505-01-01-00000.nc'
        wiso_runoff= .true.
    ''')

In [13]:
!./preview_namelists

Setting resource.RLIMIT_STACK to -1 from (307200000, -1)
Creating component namelists
  2023-04-25 15:19:23 atm
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/cam//cime_config/buildnml
     ...calling cam buildcpp to set build time options
CAM namelist copy: file1 /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/Buildconf/camconf/atm_in file2 /glade/scratch/fengzhu/CESM_output/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/run/atm_in 
  2023-04-25 15:19:24 lnd
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/clm//cime_config/buildnml
     ...calling clm buildcpp to set build time options
  2023-04-25 15:19:25 ice
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/cice//cime_config/buildnml
     ...buildnml calling cice buildcpp to set build time options
  2023-04-25 15:19:25 ocn
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/pop//cime_config/buildnml
     .

In [14]:
!qcmd -A $PROJECT -- ./case.build || exit 3

Submitting command to PBS using account P93300324:
    ./case.build

Requested custom PBS options:
    -A P93300324 

Waiting for job 9784085.chadmin1.ib0.cheyenne.ucar.edu to start ... 


Building case in directory /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301
sharedlib_only is False
model_only is False
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Generating component namelists as part of build
Creating component namelists
  2023-04-25 15:20:15 atm
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/cam//cime_config/buildnml
     ...calling cam buildcpp to set build time options
CAM namelist copy: file1 /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/Buildconf/camconf/atm_in file2 /glade/scratch/fengzhu/CESM_output/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/run/atm_in 
  2023-04-25 15:20:16 lnd
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/components/clm//cime_config/buildnml
   

In [15]:
!./xmlchange STOP_OPTION=nyears,STOP_N=1,REST_N=1,RESUBMIT=0,JOB_QUEUE=regular
!./xmlchange JOB_WALLCLOCK_TIME=12:00:00

In [16]:
!./case.submit

File /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/LockedFiles/env_batch.xml has been modified
  found difference in USER_REQUESTED_WALLTIME : case '12:00:00' locked ''
  found difference in USER_REQUESTED_QUEUE : case 'regular' locked ''

env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Creating batch scripts
Writing case.run script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_hires_fzhu/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
copying /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/env_batch.xml -> /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g17.icesm13_ihesp.PI.301/LockedFiles
copying /glade/u/home/fengzhu/CESM/cases/b.e13.B1850C5.ne16_g1

In [17]:
!qstat -u fengzhu